In [ ]:
import torch
from transformer import vit_base_patch16_224_in21k
import torchvision.transforms as transforms
import numpy as np
from flask import Flask, jsonify, app, request
from PIL import Image
from torch import softmax
import base64
import io

app = Flask(__name__)

weights_path = "model/output/transformer_zqg.pt"

# 定义预处理函数，将图片进行resize和归一化等操作
preprocess = transforms.Compose(
    [  # 数据处理
        transforms.ToTensor(),  # 转换为张量
        transforms.Resize((224, 224)),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # 归一化处理
    ]
)
# 创建模型实例
model = vit_base_patch16_224_in21k(num_classes=20, has_logits=False)
# 从.pt文件中加载权重参数
weights = torch.load(weights_path)
# 应用权重参数到模型实例
model.load_state_dict(weights)
# 获取模型实例
model = model.eval()
# 将模型转移至GPU上（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

class_names = [
    "土豆",
    "茄子",
    "香菜",
    "玉米",
    "青椒",
    "韭菜",
    "豌豆",
    "梨子",
    "苹果",
    "西兰花",
    "蒜苗",
    "水稻",
    "小麦",
    "胡萝卜",
    "葡萄",
    "南瓜",
    "黄瓜",
    "香蕉",
    "西红柿",
    "西瓜",
]

In [ ]:
img_path = (
    "dataset\\train\\葡萄\\【AI数字艺术】粉红色背景下的葡萄特写镜头VCG211447985488.jpg"
)



# @app.route("/predict", methods=["POST"])

# def predict():
print("如下是请求内容：")
print(request.get_json())

img_base64 = request.get_json()["imgdata"]

image = base64.b64decode(img_base64)

image = Image.open(io.BytesIO(image))

image_tensor = preprocess(image).unsqueeze(0).to(device)

with torch.no_grad():

    pre = model(image_tensor)
    print(pre)

    pre = softmax(pre, dim=1).detach().cpu().numpy()[0]
    print(pre)

print(np.argsort(pre))

y_hat = np.argsort(pre)[::-1][:5]

print(y_hat)

result = []

for y in y_hat:

    name = class_names[y]
    print(name)

    score = str(pre[y])
    print(score)

    result.append([name, score])

print(result)


#     return jsonify(

#         {"eror_code": "undefiened", "error_msg": "未知错误", "results": result}
#     )



# # 启动服务器

# if __name__ == "__main__":

#     app.run(host="0.0.0.0", port=5000)



# # 使用模型进行预测，并显示预测结果

# with torch.no_grad():

#     output = model(image_tensor)

#     _, predicted = torch.max(output, 1)

#     class_name = class_names[predicted.item()]